# ✋ 손 제스처 분류 모델 사전학습


## 📘 1. 프로젝트 개요

이 노트북은 손 제스처 인식 모델을 학습하기 위한 사전 작업을 수행하는 목적을 가진다.  
목표는 아래와 같다:

- 다양한 손 제스처(예: fist, open_palm, ok, heart 등)를 정확하게 분류할 수 있는 모델 학습
- **Webcam 기반 실시간 입력**을 지원하는 경량 모델로 구성
- 실시간/오프라인 예측 모두 대응

## 📸 2. 데이터 수집 방법

### ✅ 방법 1: 이미지 기반 수집 (사진 저장)

- **설명**: 웹캠으로 손 제스처를 촬영하여 `.jpg` 형식으로 저장
- **디렉토리 구조**: `dataset/{label_name}/image.jpg`

**장점**:
- 직관적이고 라벨 관리가 쉬움
- 시각적 확인이 가능함

**단점**:
- 손 위치/각도에 따라 좌표가 달라져서 학습이 불안정할 수 있음
- 배경, 조명 변화에 민감

---

### ✅ 방법 2: Mediapipe 기반 관절 특징 추출 + CSV 저장

- **설명**: Mediapipe로 손 관절 21개의 위치를 추출하고, 각도/거리/벡터 방향 등 특징을 계산하여 `.csv`로 저장

**장점**:
- 배경/조명/회전에 덜 민감
- 실시간 모델 구성에 매우 유리 (벡터화된 feature 사용)
- 매우 가볍고 빠름

**단점**:
- 데이터 시각화가 어려움
- 라벨링 오류 시 추적 어려움

---

### ✅ 방법 3: 외부 데이터셋 사용

- 예: Bria Hand Gesture Dataset, Jester Dataset, ASL Alphabet Dataset

**장점**:
- 대량의 라벨링된 고품질 이미지 확보 가능
- 학습 시간 절약

**단점**:
- 목적에 맞지 않는 라벨 구조일 수 있음
- 데이터 전처리 필요
- 모델 커스터마이징 어려움

## ✋ 3. 제스처 라벨 구성 예시

| 라벨 이름 | 설명 |
|-----------|------|
| `fist` | 주먹 쥔 손 |
| `open_palm` | 손을 편 상태 |
| `ok` | OK 제스처 (엄지+검지 동그라미) |
| `heart` | 손가락 하트 |
| `victory` | V자 손 |
| `promise` | 새끼손가락 걸기 |
| `bad` | 엄지 아래로 |
| `good` | 엄지 위로 |
| `none` | 아무 동작이 없는 상태 (오류 방지용)

## ⚙️ 4. 특징 추출 방식 비교

| 방식 | 설명 | 장점 | 단점 |
|------|------|------|------|
| x, y 좌표 원본 | Mediapipe에서 받은 raw 좌표 | 구현이 간단함 | 위치/크기/회전에 민감 |
| 상대 좌표 (벡터) | landmark 간 차이벡터 | 회전에 다소 강함 | 손이 많이 틀어지면 여전히 불안정 |
| 각도 기반 | 세 관절로 이루어진 각도 사용 | 회전/크기 변화에 매우 강함 | 계산이 필요함 |
| 거리 기반 | 손목-손끝, 손끝-손끝 거리 | 손 크기 등 비례 정보 잘 반영 | normalization 필요 |
| 벡터 방향 | 방향성 표현 | 실시간 분류에 유리 | noise에 민감할 수 있음

👉 최종 모델에서는 **각도 + 거리 + 벡터 방향 조합**을 사용하는 것이 가장 안정적

## 🧪 5. 학습 방식

### ✅ KNN (추천 ⭐)

- `scikit-learn` 기반으로 학습
```python
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=3)
model.fit(X_train, y_train)
```

**장점**:
- 간단하고 빠름
- 소량 데이터로도 잘 작동

**단점**:
- 데이터가 많아지면 느려짐
- 고차원일수록 거리 계산 정확도 저하 가능

---

### ✅ MLP / SVM / RandomForest

- Scikit-learn 또는 PyTorch 기반
- 분류 정밀도 상승
- 데이터가 많을 경우 추천

---

### ✅ CNN (이미지 기반만 해당)

- 이미지 원본을 학습하는 경우 사용
- MobileNet, EfficientNet 등 사전학습 모델 가능

**단점**:
- 학습 시간/리소스 필요
- 손만 잘라내는 전처리 필요

## 🎯 6. 평가 방식

- `train/val/test` 비율로 나눠서 평가
- 정확도(`accuracy`), 정밀도(`precision`), 재현율(`recall`) 등 활용
- Confusion Matrix 시각화 추천

## 🛠️ 7. 실시간 예측

- mediapipe + opencv + 학습된 모델로 실시간 카메라 입력 처리
- 추론 속도가 중요한 경우 → 특징 기반 모델 추천
- 예측 결과를 `cv2.putText()`로 화면에 표시 가능

## 📂 8. 파일 구조 예시

```
├── 손 제스처 분류 모델 사전학습.ipynb
├── dataset/
│   ├── fist/
│   ├── ok/
│   └── ...
├── gesture_train.csv
├── models/
│   ├── knn_model.pkl
│   └── ...
```

## 📌 9. 요약

| 항목 | 추천 방법 |
|------|-----------|
| 수집 방식 | Mediapipe로 관절 추출 + CSV 저장 |
| 특징 | 각도 + 거리 + 벡터 방향 조합 |
| 모델 | KNN 또는 RandomForest |
| 실시간 용도 | 특징 기반 분류 모델 |
| none 라벨 | 실시간 분류일 경우 필수 |
